###  Zero Copy
#### 1. **通过`FileChannel`的`transfer`实现内核态和用户态之间的零拷贝**    
虚拟存储技术的出现,使进程之间占用的内存不会互相干扰, 为了进一步增加安全性, 内存被划分为了内核空间和用户空间, 所有IO设备中的数据都要先复制到内核空间, 再有内核空间复制到用户空间后才能使用. 如果要在两个IO设备之间传输数据, 且其中至少有一个IO源是磁盘上的文件, 则可以使用`FileChannel`的`transferTo`或`transferFrom`, 让数据直接在内存空间上流转, 而不用经过用户空间产生多余的拷贝

1. `transferFrom()`  
如下打开两个文件channel,把数据从一个文件写入到另一个文件.
 下面的position和count用来描述源文件.
    * position: 从toChannel的哪个位置开始写
    * count: 从fromChannel中读多少数据写入 
    * 如果FileChannel的transferFrom是SocketChannel,且将socket设置为非阻塞模式, 那么当调用transferFrom()方法时, 会立刻把数据从socket的缓冲区中读取出来; 即便读取的大小没到count指定的大小, 且后续仍会有数据从socket中来, 也不能再次写入到FileChannel中
     <img src="img/transforfrom.png" width="70%">

2. `transferTo()`   
 同上述功能一样, 将数据从fromChannel拷贝到toChannel
    * position: 从fromChannel的那个位置开始读
    * count: 从fromChannel中读多少数据
    * 如果fromChannel是SocketChannel,且将socket设置为非阻塞模式,一样会存在transferFrom的问题: 立刻读取, 且后续即使有数据到来也不能继续读取
    <img src="img/transforto.png" width="70%">
        
        
#### 2. **传统的数据传输方式**  
  像这种从文件读取数据然后将数据通过网络传输给其他的程序的方式其核心操作就是如下两个调用：
```java
File.read(fileDesc,buf,len);
Socket.send(socket,buf,len);
```
其上操作看上去只有两个简单的调用，但是其内部过程却要经历四次用户态和内核态的切换以及四次的数据复制操作：
<img src="img/transfor3.png" width="60%">  
上图展示了数据从文件到socket的内部流程。   
下面看下用户态和内核态的切换过程：   
<img src="img/trans5.png" width="70%">  
步骤如下：
1. read()的调用引起了从用户态到内核态的切换（看图二），内部是通过sys_read()（或者类似的方法）发起对文件数据的读取。数据的第一次复制是通过DMA（直接内存访问）将磁盘上的数据复制到内核空间的缓冲区中；
2. 数据从内核空间的缓冲区复制到用户空间的缓冲区后，read()方法也就返回了。此时内核态又切换回用户态，现在数据也已经复制到了用户地址空间的缓存中；
3. socket的send()方法的调用又会引起用户态到内核的切换，第三次数据复制又将数据从用户空间缓冲区复制到了内核空间的缓冲区，这次数据被放在了不同于之前的内核缓冲区中，这个缓冲区与数据将要被传输到的socket关联；
4. send()系统调用返回后，就产生了第四次用户态和内核态的切换。随着DMA单独异步的将数据从内核态的缓冲区中传输到协议引擎发送到网络上，有了第四次数据复制。


#### 3. **zero copy的数据传输方式**  
zero copy主要用到的技术是: 
* DMA数据传输 
* 内存地址映射  

下图展示了通过transferTo实现数据传输的路径：
<img src="img/trans1.png" width="50%">  
下图展示了内核态、用户态的切换情况：
<img src="img/trans2.png" width="60%">  
使用transferTo()方式所经历的步骤：
1. transferTo调用会引起DMA将文件内容复制到读缓冲区(内核空间的缓冲区)，然后数据从这个缓冲区复制到另一个与socket输出相关的内核缓冲区中； 
2. 第三次数据复制就是DMA把socket关联的缓冲区中的数据复制到协议引擎上发送到网络上。  

这次改善，我们是通过将内核、用户态切换的次数从四次减少到两次，将数据的复制次数从四次减少到三次(只有一次用到cpu资源)。但这并没有达到我们零复制的目标。如果底层网络适配器支持收集操作的话，我们可以进一步减少内核对数据的复制次数。在内核为2.4或者以上版本的linux系统上，socket缓冲区描述符将被用来满足这个需求。这个方式不仅减少了内核用户态间的切换，而且也省去了那次需要cpu参与的复制过程。从用户角度来看依旧是调用transferTo()方法，但是其本质发生了变化：
1. 调用transferTo方法后数据被DMA从文件复制到了内核的一个缓冲区中；
2. 数据不再被复制到socket关联的缓冲区中了，仅仅是将一个描述符（包含了数据的位置和长度等信息）追加到socket关联的缓冲区中。DMA直接将内核中的缓冲区中的数据传输给协议引擎，消除了仅剩的一次需要cpu周期的数据复制。